## Import Packages

In [1]:
import pandas as pd
import numpy as np

## Import Data and prepare data

In [13]:
# Just remember to replace the file paths for the paths in your computer

# get inflation rate data
data_inf = pd.read_csv(r"D:\Work\Random Work\miro_inf_data.csv")


# get sector returns data
data_sectors = pd.read_csv(r"D:\Work\Random Work\miro_sector_data.csv")


# get benchmark returns data
data_bench = pd.read_csv(r"D:\Work\Random Work\miro_benchmark_data.csv")


# set date column as index for both dataframes
data_sectors.set_index('date', inplace=True)
data_bench.set_index('date', inplace=True)
data_inf.set_index('date', inplace=True)

# set indeces as date_time 
data_inf['date'] = pd.to_datetime(data_inf['date'] )
data_sectors['date'] = pd.to_datetime(data_sectors['date'] )
data_bench['date'] = pd.to_datetime(data_bench['date'] )

# calculate excess returns vs benchmark
data_sectors_xsret = data_sectors.subtract(data_bench['benchmark'], axis=0)

# print to double check
print(data_sectors_xsret)
print(data_inf)



              energy  materials  industrials   condisc        it  financials  \
date                                                                           
1973-02-01 -0.037876   0.008030     0.009140 -0.008433 -0.020299    0.065328   
1973-03-01  0.036912   0.028578    -0.000215 -0.014238 -0.009986   -0.021737   
1973-04-01  0.045699  -0.012418    -0.020137  0.001772 -0.005158   -0.041807   
1973-05-01 -0.008219   0.002588     0.014614 -0.008279 -0.016681    0.004334   
1973-06-01  0.022831  -0.003215    -0.002445 -0.007234  0.002996    0.023299   
...              ...        ...          ...       ...       ...         ...   
2022-09-01  0.000915   0.012892    -0.008601 -0.002618 -0.028263    0.014258   
2022-10-01  0.119707  -0.013322     0.037646 -0.065047  0.009404    0.017651   
2022-11-01 -0.041313   0.062447     0.014885 -0.004790 -0.004861    0.009572   
2022-12-01  0.003618   0.012660     0.018759 -0.031327 -0.039224    0.015212   
2023-01-01 -0.041629   0.027462    -0.01

## Divide inflation into quartiles

In [14]:
# calculate quartile cutoffs
q1_cutoff = data_inf['inflation_rate'].quantile(0.25)
q2_cutoff = data_inf['inflation_rate'].quantile(0.5)
q3_cutoff = data_inf['inflation_rate'].quantile(0.75)

# create function to classify periods into quartiles
def classify_inflation(row):
    if row['inflation_rate'] <= q1_cutoff:
        return '1st quartile'
    elif row['inflation_rate'] <= q2_cutoff:
        return '2nd quartile'
    elif row['inflation_rate'] <= q3_cutoff:
        return '3rd quartile'
    else:
        return '4th quartile'

# apply function to create new column with quartile classification
data_inf['inflation_regime'] = data_inf.apply(classify_inflation, axis=1)

# print to check
print(data_inf)

            inflation_rate inflation_regime
date                                       
1973-02-01        3.864734     3rd quartile
1973-03-01        4.830918     3rd quartile
1973-04-01        5.301205     4th quartile
1973-05-01        5.528846     4th quartile
1973-06-01        5.995204     4th quartile
...                    ...              ...
2022-09-01        8.214854     4th quartile
2022-10-01        7.762493     4th quartile
2022-11-01        7.135348     4th quartile
2022-12-01        6.444940     4th quartile
2023-01-01        6.347156     4th quartile

[600 rows x 2 columns]


## Calculate average returns by inflation regime

In [25]:
# merge inflation and sector returns dataframes
merged_data = pd.merge(left=data_sectors_xsret,right=data_inf, left_index=True, right_index=True)


# group returns by sector and inflation regime and calculate mean return for each group
grouped_data= merged_data.groupby(['inflation_regime']).mean().drop(columns=['inflation_rate'])


# group data by inflation regime and calculate annualized mean for each sector
sector_returns_by_regime = merged_data.groupby('inflation_regime').mean() * 12

# print results
print(sector_returns_by_regime)

                    energy  materials  industrials   condisc        it  \
inflation_regime                                                         
1st quartile     -0.057927   0.003930    -0.016538  0.033005  0.053479   
2nd quartile      0.031052  -0.003958    -0.005473 -0.009037  0.069646   
3rd quartile      0.026327   0.029855     0.000124 -0.013674 -0.051806   
4th quartile      0.060178  -0.023060     0.013429 -0.021619  0.004192   

                  financials  constaples  commserv  healthcare  utilities  \
inflation_regime                                                            
1st quartile        0.001124   -0.026377  0.007374   -0.007314  -0.037712   
2nd quartile       -0.020617   -0.010268  0.013461    0.022860  -0.040323   
3rd quartile        0.028704    0.046337 -0.050462    0.029489   0.047405   
4th quartile       -0.001138   -0.008956  0.001744    0.026203   0.023740   

                  inflation_rate  
inflation_regime                  
1st quartile          